In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
!pip install transformers
!pip install gradio

In [19]:
import torch
import gradio as gr
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [20]:
model_path = "/content/drive/MyDrive/LS_FINAL_SUBMISSION/chatbot_model.pth"

In [35]:
tokenizer_name = "gpt2"

# Loading the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_name)
tokenizer.add_special_tokens({"pad_token": "<pad>",
                              "bos_token": "<startofstring>",
                              "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<bot>:"])

# Setting the vocabulary size to match the tokenizer
model = GPT2LMHeadModel.from_pretrained(tokenizer_name)
model.resize_token_embeddings(len(tokenizer))

# Loading the fine-tuned model's state dictionary
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

# Setting the model to evaluation mode
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50261, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50261, bias=False)
)

In [48]:
import gradio as gr
import torch

# Function to generate a chatbot response based on user input
def generate_response(input_text):

    # Preprocessing the text
    input_text = "<startofstring> " + input_text + " <bot>: "
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    input_ids = input_ids.to(torch.device('cpu'))  # Move to CPU for inference
    attention_mask = torch.ones_like(input_ids)

    with torch.no_grad():
        output = model.generate(input_ids, attention_mask=attention_mask)

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    g = f(generated_text)
    if(g==""):
      return generated_text
    else:
      return g

# Create the Gradio interface
iface = gr.Interface(
    fn=generate_response,
    inputs=gr.inputs.Textbox(),
    outputs=gr.outputs.Textbox(label="Chatbot's Response"),
    title="Chatbot",
    description="Engage in a conversation with the chatbot!",
    theme="huggingface",  # Use Hugging Face theme for a consistent look
    examples=[
        ["Hello how are you?"],
        ["Who are you?"],
        ["Can you swim?"],
    ],
)

# Launch the interface
iface.launch()


<ipython-input-48-e9d198d62a07>:24: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Textbox(),
<ipython-input-48-e9d198d62a07>:24: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(),
<ipython-input-48-e9d198d62a07>:24: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(),
<ipython-input-48-e9d198d62a07>:25: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=gr.outputs.Textbox(label="Chatbot's Response"),
/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:699: UserWarning: Cannot load huggingface. Caught Exception: The space huggingface does not exist
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [47]:
## Every output it generates is of some form --- <bot>: ------
import regex as re


def f(inp):
  pattern= r"<bot>: ([a-zA-Z\s0-9[:punct:]]*)"
  result = re.search(pattern,inp)
  if result:
    sentence = result.group(1)
    return (sentence)
  else:
    return ""

f("Can you swim? <bot>: Yes, i swim in the ocean, and surf")

'Yes, i swim in the ocean, and surf'